In [4]:
import keras
import cv2
from skimage.transform import rotate, resize, SimilarityTransform, warp
import os
import itertools
import numpy as np
import random as rand
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten, Dense
from keras.utils.training_utils import multi_gpu_model
from sklearn import metrics
from sklearn.metrics import precision_recall_curve, accuracy_score, precision_score, recall_score
from sklearn.metrics import average_precision_score, confusion_matrix
%matplotlib inline

In [5]:
MODEL_PATH = "./binary_classifier/net_2_model.h5"

In [6]:
model = keras.models.load_model(MODEL_PATH)

ImportError: `load_model` requires h5py.